In [1]:
import geohash
from datetime import datetime as dt
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
# import seaborn as sns
import os
import time
# import tensorflow as tf
# import utm
import gc
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.cluster.hierarchy import dendrogram, linkage
# from statsmodels.tsa.tsatools import lagmat
from functools import reduce
from sklearn.linear_model import HuberRegressor, LinearRegression

# sns.set_style("darkgrid")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%matplotlib inline

In [2]:
# import lightgbm as lgb
# import xgboost as xgb

In [32]:
this_dir = os.path.abspath('')
data_path = os.path.join(this_dir, "training.csv")
data = pd.read_csv(data_path)

In [33]:
feat_dir = os.path.join(this_dir, "new_feats_13_June")

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4206321 entries, 0 to 4206320
Data columns (total 4 columns):
geohash6     object
day          int64
timestamp    object
demand       float64
dtypes: float64(1), int64(1), object(2)
memory usage: 128.4+ MB


In [36]:
data['time'] = pd.to_datetime(data['timestamp'], format = '%H:%M').dt.time

In [37]:
data.drop(['timestamp'], axis=1, inplace = True)

In [8]:
day_split = 46

In [9]:
data_train = data.loc[data.day <= day_split].copy()

In [38]:
# data_val = data.loc[(day_split < data.day)&(data.day<=day_split+14)].copy()
data_val = data.loc[day_split < data.day].copy()

In [46]:
type(data_val['time'])

pandas.core.series.Series

In [ ]:
data_test = data.loc[((day_split+14) < data.day) & (data.day<=day_split+28)].copy() 

In [ ]:
data_test.day.nunique()

In [ ]:
# data_train['day'] = data_train['day'].apply(lambda x: x%31 if x < 31 else 31)

In [ ]:
data_train.head()

### FILL NANs

In [10]:
data_train  = data_train.set_index(['day', 'time', 'geohash6']).unstack()

In [11]:
data_train.head()

demand                                                           \
geohash6     qp02yc qp02yf qp02yv qp02yy qp02yz qp02z1 qp02z3 qp02z4  qp02z5   
day time                                                                       
1   00:00:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN   
    00:15:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  0.0006   
    00:30:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN   
    00:45:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN   
    01:00:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN   

                                                                              \
geohash6     qp02z6 qp02z7    qp02z9    qp02zc    qp02zd    qp02ze    qp02zf   
day time                                                                       
1   00:00:00    NaN    NaN       NaN       NaN  0.022396       NaN       NaN   
    00:15:00    NaN    NaN       NaN  0.038979       NaN  0.010554       NaN   
    00:30:00    NaN    NaN  0.022022       NaN       NaN       NaN       NaN   
    00:45:00    NaN    NaN       NaN       NaN  0.070525  0.001295       NaN   
    01:00:00    NaN    NaN  0.083947       NaN  0.045482  0.047912  0.006503   

                                                                               \
geohash6     qp02zg    qp02zh qp02zj    qp02zk    qp02zm qp02zn qp02zp qp02zq   
day time                                                                        
1   00:00:00    NaN       NaN    NaN       NaN       NaN    NaN    NaN    NaN   
    00:15:00    NaN       NaN    NaN       NaN       NaN    NaN    NaN    NaN   
    00:30:00    NaN       NaN    NaN       NaN       NaN    NaN    NaN    NaN   
    00:45:00    NaN  0.008541    NaN       NaN       NaN    NaN    NaN    NaN   
    01:00:00    NaN       NaN    NaN  0.049005  0.047575    NaN    NaN    NaN   

                                                                          \
geohash6        qp02zr    qp02zs    qp02zt    qp02zu    qp02zv    qp02zw   
day time                                                                   
1   00:00:00  0.042046       NaN  0.001112  0.001831  0.006886  0.027161   
    00:15:00  0.007143       NaN       NaN  0.004774       NaN  0.043129   
    00:30:00       NaN       NaN       NaN  0.009024       NaN  0.013028   
    00:45:00  0.060482       NaN       NaN  0.002197       NaN  0.042912   
    01:00:00  0.009577  0.029216       NaN       NaN       NaN       NaN   

                                                                          \
geohash6        qp02zx    qp02zy    qp02zz    qp03jq    qp03jr    qp03jw   
day time                                                                   
1   00:00:00  0.043180       NaN  0.176994  0.009146  0.019114  0.012002   
    00:15:00  0.147283       NaN  0.141265  0.004957  0.005584       NaN   
    00:30:00  0.148128  0.119734  0.254303  0.002096  0.015964       NaN   
    00:45:00  0.125946  0.104908  0.255470  0.007507  0.015012  0.005148   
    01:00:00  0.127945  0.128632  0.161297       NaN  0.012000       NaN   

                                                                         \
geohash6       qp03jx    qp03jy    qp03jz    qp03m2    qp03m3    qp03m6   
day time                                                                  
1   00:00:00      NaN  0.000067  0.015054       NaN       NaN       NaN   
    00:15:00      NaN  0.016033  0.022844       NaN       NaN  0.008224   
    00:30:00      NaN  0.003845  0.001576       NaN       NaN  0.009145   
    00:45:00  0.01036  0.001794  0.010668  0.000076       NaN  0.013351   
    01:00:00      NaN  0.002844  0.030557  0.010522  0.000934  0.034528   

                                                                          \
geohash6        qp03m7    qp03m8    qp03m9    qp03mb    qp03mc    qp03md   
day time                                                                   
1   00:00:00       NaN       NaN   

In [12]:
data_train = data_train.fillna(0)
data_train = data_train.stack(dropna=False)

In [13]:
data_train = data_train.reset_index().sort_values(by = ['geohash6', 'day','time']).reset_index(drop=True)

In [15]:
data_train['day_mod'] = data_train['day'].apply(lambda x: x%31 if x < 31 else 31)

In [16]:
data_train.head()

,day,time,geohash6,demand,day_mod
0,1,00:00:00,qp02yc,0.0,1
1,1,00:15:00,qp02yc,0.0,1
2,1,00:30:00,qp02yc,0.0,1
3,1,00:45:00,qp02yc,0.0,1
4,1,01:00:00,qp02yc,0.0,1


## Find consecutive values of 0s

In [ ]:
# data_train['demand_grp'] = (data_train.demand.shift(1) == 0).astype('float').cumsum()
# # # data_train['demand_mask'] = data_train.demand.ne(0)

In [17]:
cons_zeros = data_train.copy()

In [18]:
g = cons_zeros['demand'].ne(cons_zeros['demand'].shift(1)).cumsum()
counts = cons_zeros.groupby(['geohash6','day',g])['demand'].cumcount() + 1
cons_zeros['consec_zeros'] = np.where(cons_zeros['demand'].eq(0), counts, 0)

#replace 0 to `NaN`s
a = cons_zeros['consec_zeros'].mask(cons_zeros['consec_zeros'].eq(0))
#add 1 to forward filling missing values by limit 1 per groups
cons_zeros['consec_zeros'] = (np.where(a.isna(), 
                               a.groupby([cons_zeros['geohash6'],cons_zeros['day']]).ffill(limit=1) + 1, 
                               cons_zeros['consec_zeros']) - 1)
cons_zeros['consec_zeros'] = cons_zeros['consec_zeros'].fillna(0).astype(int)

In [19]:
cons_zeros['consec_zeros'] = cons_zeros['consec_zeros'].astype(float)

In [21]:
cons_zeros.drop(['day'],axis=1, inplace=True)

In [22]:
cons_zeros.to_csv('./test_cons_zeros.csv', index=False)

In [ ]:
data_train['consec_zeros'] = cons_zeros['consec_zeros']

In [ ]:
# # g = (data_train.demand.shift(1) != 0).astype('float').cumsum()
# g = data_train['demand'].ne(data_train['demand'].shift(1)).cumsum()
# # g = data_train.groupby((data_train['demand'] != data_train['demand'].shift(1)).cumsum()).cumcount()
# counts = data_train.groupby(['geohash6','day',g])['demand'].cumcount() + 1
# data_train['consec_zeros'] = np.where(data_train['demand'].eq(0), counts, 0)

In [ ]:
# #replace 0 to `NaN`s
# a = data_train['consec_zeros'].mask(data_train['consec_zeros'].eq(0))

In [ ]:
# #add 1 to forward filling missing values by limit 1
# data_train['consec_zeros'] = np.where(a.isna(), a.ffill(limit=1) + 1, data_train['consec_zeros']) - 1
# data_train['consec_zeros'] = data_train['consec_zeros'].fillna(0).astype(int)

In [ ]:
cons_zeros.head()

In [ ]:
cons_zeros = cons_zeros.drop('demand', axis=1)

In [ ]:
data_train.iloc[:5]

In [ ]:
data_train.iloc[2965:2970]

In [ ]:
data_train['week_day'] = data_train['day'].apply(lambda x: x%7)

In [ ]:
data_train['hour'] = data_train['time'].apply(lambda x: x.hour)
data_train['minute'] = data_train['time'].apply(lambda x: x.minute)

## Find mean demand in one hour according to geo and week

In [ ]:
mean_time_interval = pd.pivot_table(data_train, index=['geohash6', 'week_day'], columns=['hour'],
                                    values='demand',
                                    aggfunc=np.mean)

In [ ]:
mean_time_interval.info()

In [ ]:
mean_demand_by_time = pd.melt(mean_time_interval.reset_index(), 
                              id_vars = ['geohash6','week_day'], 
                              value_name = 'mean_demand_per_hour', )

In [ ]:
mean_demand_by_time.head()

In [ ]:
mean_week_interval = pd.pivot_table(data_train, index=['geohash6'], columns=['week_day'],
                                    values='demand',
                                    aggfunc=np.mean)

In [ ]:
mean_demand_by_week = pd.melt(mean_week_interval.reset_index(), 
                              id_vars = ['geohash6'], 
                              value_name = 'mean_demand_per_week', )

In [ ]:
mean_geohour_interval = pd.pivot_table(data_train, index=['geohash6'], columns=['hour'],
                                    values='demand',
                                    aggfunc=np.mean)

In [ ]:
mean_demand_by_geo = pd.melt(mean_geohour_interval.reset_index(), 
                              id_vars = ['geohash6'], 
                              value_name = 'mean_demand_per_geo', )

In [ ]:
mean_demand_by_week.head()

In [ ]:
# mean_demand_by_geo.head()

In [ ]:
31%31.0

### Find median demand in one hour by time and week

In [ ]:
# std_time_interval = pd.pivot_table(data_train, index=['geohash6', 'week_day'], columns=['hour'], 
#                                    values='demand',
#                                    aggfunc=lambda x: np.std(x, ddof=0))

std_time_interval = pd.pivot_table(data_train, index=['geohash6', 'week_day'], columns=['hour'], 
                                   values='demand',
                                   aggfunc=lambda x: np.median(x))

In [ ]:
std_time_interval.head()

In [ ]:
std_demand_by_time = pd.melt(std_time_interval.reset_index(), 
                             id_vars = ['geohash6','week_day'], 
                             value_name = 'median_demand_per_hour',)
#                              var_name = ['var_demand','hour'])

In [ ]:
std_week_interval = pd.pivot_table(data_train, index=['geohash6'], columns=['week_day'], 
                                   values='demand',
                                   aggfunc=lambda x: np.median(x))

In [ ]:
std_demand_by_week = pd.melt(std_week_interval.reset_index(), 
                             id_vars = ['geohash6'], 
                             value_name = 'median_demand_per_week',)

In [ ]:
std_geohour_interval = pd.pivot_table(data_train, index=['geohash6'], columns=['hour'], 
                                   values='demand',
                                   aggfunc=lambda x: np.median(x))

In [ ]:
std_demand_by_geo = pd.melt(std_geohour_interval.reset_index(), 
                             id_vars = ['geohash6'], 
                             value_name = 'median_demand_per_geo',)

In [ ]:
# std_demand_by_geo.head()

In [ ]:
# std_demand_by_week.head()

#### Join mean and standard deviation

In [ ]:
dfs = [data_train, mean_demand_by_time, std_demand_by_time]

In [ ]:
data_mean_std = reduce(lambda left,right: pd.merge(left,right,on=['geohash6', 'week_day', 'hour']), dfs)

In [ ]:
del dfs
gc.collect()

In [ ]:
dfs_week = [data_mean_std,mean_demand_by_week, std_demand_by_week]

In [ ]:
data_mean_std = reduce(lambda left,right: pd.merge(left,right,on=['geohash6', 'week_day']), dfs_week)

In [ ]:
del dfs_week
gc.collect()

In [ ]:
dfs_geo = [data_mean_std,mean_demand_by_geo, std_demand_by_geo]

In [ ]:
data_mean_std = reduce(lambda left,right: pd.merge(left,right,on=['geohash6', 'hour']), dfs_geo)

In [ ]:
del dfs_geo
gc.collect()

In [ ]:
data_mean_std.head()

In [ ]:
std_demand_by_time['geohash6'].nunique()

In [ ]:
data_train = data_mean_std.copy()

In [ ]:
del data_mean_std
gc.collect()

### Create lag and rolling variable of demand

In [ ]:
# data_train = data_train.sort_values(by = ['geohash6', 'day','time'])

In [ ]:
# data_train = data_train.reset_index(drop=True)

In [ ]:
# lag = [-1,-5]
# tmp = data_train[['day','time', 'geohash6', 'demand']].copy()
# tmp = tmp.groupby('geohash6')['demand']
# for i in lag:
# #     data_train["lag_demand_%d" % i] =  tmp.transform(lambda x: x.shift(i))
#     data_train["lag_diff_%d" %i] = tmp.transform(lambda x: (x - x.shift(i)))
#     data_train["lag_diff_%d" %i] = data_train["lag_diff_%d" %i].fillna(0)
# #     data_train["lag_demand_%d" % i] = data_train["lag_demand_%d" % i].fillna(0)
# #     tmp["lag_demand_%d" % i] = data_train["lag_demand_%d" % i]
#     print(i)

In [ ]:
# rolling = [6]
# for i in rolling:
# #     data_train["rolling_mean_%d" % i] =  tmp.transform(lambda x: x.rolling(i).mean())
#     data_train["rolling_u_diff_%d" % i] =  tmp.transform(lambda x: x-x.rolling(i).mean())
    
# #     data_train["rolling_mean_%d" % i] = data_train["rolling_mean_%d" % i].fillna(0)
#     data_train["rolling_u_diff_%d" % i] = data_train["rolling_u_diff_%d" % i].fillna(0)
    
# #     data_train["rolling_median_%d" % i] =  tmp.transform(lambda x: x.rolling(i).median())
#     data_train["rolling_m_diff_%d" % i] =  tmp.transform(lambda x: x-x.rolling(i).median())
    
# #     data_train["rolling_median_%d" % i] =data_train["rolling_median_%d" % i].fillna(0)
#     data_train["rolling_m_diff_%d" % i] = data_train["rolling_m_diff_%d" % i].fillna(0)
    
# #     tmp["rolling_mean_%d" % i] = data_train["rolling_mean_%d" % i]
# #     tmp["rolling_median_%d" % i] = data_train["rolling_median_%d" % i]
#     print(i)
# del tmp
# gc.collect()

In [ ]:
data_train['latitude'] = data_train['geohash6'].apply(lambda x: geohash.decode(x)[0])
data_train['longitude'] = data_train['geohash6'].apply(lambda x: geohash.decode(x)[1])

In [ ]:
data_train.head()

In [ ]:
data_train.tail(1)

In [ ]:
# data_train['u_hour_diff'] = data_train['demand'] - data_train['mean_demand_per_hour']
# data_train['med_hour_diff'] = data_train['demand'] - data_train['median_demand_per_hour']

In [ ]:
# data_train.to_csv('./train_new_mod_31.csv', index = False)

In [ ]:
# data_train = pd.read_csv('./train.csv')

In [ ]:
# data_train.to_csv(os.path.join(feat_dir, "train.csv"), index=False)

#### More fetures: Lagging, Weekly autocorrelation, Time-Period (morning, afternoon, evening) autocorrelation, Rolling mean average

## Prepare val data

In [39]:
data_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1061208 entries, 7 to 4206314
Data columns (total 4 columns):
geohash6    1061208 non-null object
day         1061208 non-null int64
demand      1061208 non-null float64
time        1061208 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 40.5+ MB


In [40]:
data_val.to_csv('./vallllll2.csv', index=False)

In [ ]:
data_val['week_day'] = data_val['day'].apply(lambda x: x%7)

In [ ]:
data_val['hour'] = data_val['time'].apply(lambda x: x.hour)
data_val['minute'] = data_val['time'].apply(lambda x: x.minute)

In [ ]:
data_val['day'] = data_val['day'].apply(lambda x: x%31)

In [ ]:
data_val = pd.merge(data_val, cons_zeros, how = 'left', on=['day','time','geohash6'])

In [ ]:
data_val.head()

In [ ]:
data_val.info()

In [ ]:
data_val.nunique()

In [ ]:
data_val = pd.merge(data_val, mean_demand_by_time, how = 'left', on = ['week_day','hour' , 'geohash6'])

In [ ]:
data_val = pd.merge(data_val, std_demand_by_time, how = 'left', on = ['week_day','hour' , 'geohash6'])

In [ ]:
data_val.head()

In [ ]:
data_val = pd.merge(data_val, std_demand_by_week, how = 'left', on = ['week_day' , 'geohash6'])

In [ ]:
data_val = pd.merge(data_val, mean_demand_by_week, how = 'left', on = ['week_day' , 'geohash6'])

In [ ]:
data_val = pd.merge(data_val, std_demand_by_geo, how = 'left', on = ['hour' , 'geohash6'])

In [ ]:
data_val = pd.merge(data_val, mean_demand_by_geo, how = 'left', on = ['hour' , 'geohash6'])

In [ ]:
data_val['latitude'] = data_val['geohash6'].apply(lambda x: geohash.decode(x)[0])
data_val['longitude'] = data_val['geohash6'].apply(lambda x: geohash.decode(x)[1])

In [ ]:
data_val.head()

In [ ]:
data_val.to_csv(os.path.join(feat_dir, "val.csv"), index=False)

## Prepare test data

In [ ]:
# data_test  = data_test.set_index(['day', 'time', 'geohash6']).unstack()

In [ ]:
# data_test = data_test.fillna(0)
# data_test = data_test.stack(dropna=False)

In [ ]:
# data_test = data_test.reset_index().sort_values("geohash6").reset_index(drop=True)

In [ ]:
data_test.head()

In [ ]:
data_test['week_day'] = data_test['day'].apply(lambda x: x%7)

In [ ]:
data_test['hour'] = data_test['time'].apply(lambda x: x.hour)
data_test['minute'] = data_test['time'].apply(lambda x: x.minute)

In [ ]:
data_test['day'] = data_test['day'].apply(lambda x: x%31)

In [ ]:
data_test = pd.merge(data_test, cons_zeros, how = 'left', on=['day','time','geohash6'])

In [ ]:
data_test.head()

In [ ]:
data_test.info()

In [ ]:
data_test.nunique()

In [ ]:
data_test = pd.merge(data_test, mean_demand_by_time, how = 'left', on = ['week_day','hour' , 'geohash6'])

In [ ]:
data_test = pd.merge(data_test, std_demand_by_time, how = 'left', on = ['week_day','hour' , 'geohash6'])

In [ ]:
data_test = pd.merge(data_test, std_demand_by_week, how = 'left', on = ['week_day' , 'geohash6'])

In [ ]:
data_test = pd.merge(data_test, mean_demand_by_week, how = 'left', on = ['week_day' , 'geohash6'])

In [ ]:
data_test = pd.merge(data_test, std_demand_by_geo, how = 'left', on = ['hour' , 'geohash6'])

In [ ]:
data_test = pd.merge(data_test, mean_demand_by_geo, how = 'left', on = ['hour' , 'geohash6'])

In [ ]:
data_test['latitude'] = data_test['geohash6'].apply(lambda x: geohash.decode(x)[0])
data_test['longitude'] = data_test['geohash6'].apply(lambda x: geohash.decode(x)[1])

In [ ]:
# data_test.to_csv('./test_new_mod_31.csv', index= False)

In [ ]:
data_test.head()

In [ ]:
data_train.head()

In [ ]:
data_test.to_csv(os.path.join(feat_dir, "test.csv"), index=False)

In [ ]:
# cols =  data_train.columns.values

In [ ]:
# data_test = data_test[cols]

In [ ]:
# data_train.head()

In [ ]:
# data_train['hour'] = data_train['hour'].astype(float)
# data_train['minute'] = data_train['minute'].astype(float)
# data_train['week_day'] = data_train['week_day'].astype(float)
# data_test['hour'] = data_test['hour'].astype(float)
# data_test['minute'] = data_test['minute'].astype(float)
# data_test['week_day'] = data_test['week_day'].astype(float)

In [ ]:
# X_train = data_train.drop(['geohash6','time','demand'],axis=1)
# y_train = data_train['demand']
# X_test = data_test.drop(['geohash6','time','demand'],axis=1)
# y_test = data_test['demand']

In [ ]:
# X_train = X_train.astype(float)
# X_test = X_test.astype(float)

In [ ]:
# lgb_train_set = lgb.Dataset(X_train,y_train)
# lgb_test_set = lgb.Dataset(X_test,y_test)

In [ ]:
# params = {
#     'nthread': 10, 
#     'max_depth': 5,
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'regression_l2',
#     'metric': 'l2', 
#     'num_leaves': 64,
#     'learning_rate': 0.2,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'lambda_l1': 3.097758978478437,
#     'lambda_l2': 2.9482537987198496,
#     'verbose': 1,
# }

In [ ]:
# eval_results = {}

In [ ]:
# %%time
# reg = lgb.train(params, lgb_train_set, 500, 
#                 valid_sets=[lgb_train_set, lgb_test_set],
#                 evals_result= eval_results,
# #                 categorical_feature=['geohash6_numeric','neighbor_1', 'neighbor_2', 'neighbor_3', 'neighbor_4'],
#                 early_stopping_rounds=50)

In [ ]:
# def get_rmse(y_true, y_pred):
#     return np.sqrt(np.mean(np.square(y_true - y_pred)))

In [ ]:
# del lr
# gc.collect()

In [ ]:
# lr = HuberRegressor()
# lr.fit(X_train, y_train)

In [ ]:
# y = lr.predict(X_test[:1000])

In [ ]:
# print(get_rmse(y_test[:1000], y))

## New feats
#### [24]	training's l2: 0.000535045	valid_1's l2: 0.0043477
#### [19]	training's l2: 0.000506763	valid_1's l2: 0.00425993 No rolling 2
#### [20]	training's l2: 0.00049237	valid_1's l2: 0.00421041
#### [19]	training's l2: 0.000914208	valid_1's l2: 0.00368118 mean and median only hour and week
#### [23]	training's l2: 0.00088711	valid_1's l2: 0.003642 mean and median only hour, week and hour in each geo

## Old way
#### [500]	training's l2: 0.00248159	valid_1's l2: 0.00439167
#### [71]	training's l2: 0.00144653	valid_1's l2: 0.00293175
#### [49]	training's l2: 0.00147436	valid_1's l2: 0.00285439
#### [54]	training's l2: 0.000842566	valid_1's l2: 0.00102703

In [ ]:
ax = lgb.plot_importance(reg, max_num_features=15, importance_type='gain')
plt.show()

In [ ]:
ax = lgb.plot_metric(eval_results, metric='l2')
plt.show()

In [ ]:
%%time
xgb_reg = xgb.XGBRegressor(n_estimators=500, eta=0.2, objective = 'reg:squarederror', booster ='dart')
xgb_reg.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_test, y_test)], 
            eval_metric = 'rmse', early_stopping_rounds=10)

## New feats
#### [28]	validation_0-rmse:0.03442	validation_1-rmse:0.058199
#### [33]	validation_0-rmse:0.033717	validation_1-rmse:0.058287

## Old way
#### [36]	validation_0-rmse:0.044193	validation_1-rmse:0.052694

#### [52]	validation_0-rmse:0.030115	validation_1-rmse:0.032055
#### [48]	validation_0-rmse:0.029997	validation_1-rmse:0.031798

In [ ]:
xgb.plot_importance(xgb_reg,importance_type='gain')
plt.show()

In [ ]:
del reg,xgb_reg
gc.collect()

## Rolling mean in each day

In [ ]:
# data_test.head()

In [ ]:
# tmp = data_test[['day','time', 'geohash6', 'demand']].copy()

In [ ]:
# tmp.head()

In [ ]:
# tmp = tmp.sort_values(by = ['geohash6', 'day','time'])

In [ ]:
# tmp = tmp.reset_index(drop=True)

In [ ]:
# tmp.head()

In [ ]:
# tmp['moving'] = tmp.groupby('geohash6')['demand'].transform(lambda x: x.rolling(2).mean())

In [ ]:
# tmp.head()